In [1]:
from collections import Counter

import categories
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_score
from sklearn.externals import joblib
from more_itertools import flatten

from ml import import_data, create_classifier

In [2]:
df = import_data("data/en.openfoodfacts.org.products.csv")
fr_df = df[df['countries_tags'] == 'en:france']
filt_df = fr_df[pd.notnull(fr_df['categories_tags'])]

/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
categories_counter = Counter(flatten(c for c in filt_df['categories_tags']))
print("Number of categories: {}".format(len(categories_counter)))
# for name, count in categories_counter.most_common(500):
#     print(f"{name} - {count}")

Number of categories: 12390


In [4]:
CATEGORY_COUNT_THRESHOLD = 25
excluded_categories = {"en:plant-based-foods-and-beverages", "en:plant-based-foods"}

selected_categories = [cat for (cat, count) in categories_counter.items() if count >= CATEGORY_COUNT_THRESHOLD and cat not in excluded_categories]
selected_categories_set = set(selected_categories)
print("Number of selected categories: {}".format(len(selected_categories_set)))

Number of selected categories: 1664


In [5]:
criterion = filt_df['categories_tags'].map(lambda x: bool(set(x).intersection(selected_categories_set)))
cat_df = filt_df[criterion]
cat_df['categories_tags_int'] = cat_df.categories_tags.apply(lambda categories: [selected_categories.index(x) for x in categories if x in selected_categories])
cat_df['categories_tags_array'] = cat_df.categories_tags.apply(lambda categories: [1 if x in categories else 0 for x in selected_categories])
print(f"{len(fr_df)} elements in original dataframe, {len(cat_df)} after category filter")

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


405973 elements in original dataframe, 120358 after category filter


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [6]:
TRAIN = False

df_train, df_test = train_test_split(cat_df, random_state=42)
y_train = np.array(list(df_train.categories_tags_array.values))
y_test = np.array(list(df_test.categories_tags_array.values))

if TRAIN:
    classifier = create_classifier()
    classifier.fit(df_train, y_train)
    joblib.dump(classifier, 'clf-model-wide.joblib')
else:
    classifier = joblib.load('clf-model-wide.joblib')

y_pred = classifier.predict(df_test)
print(classification_report(y_test, y_pred, target_names=selected_categories))

                                                                                                   precision    recall  f1-score   support

                                                                                          en:pies       0.85      0.33      0.48       132
                                                                                    en:sweet-pies       1.00      0.14      0.24        44
                                                             en:fruits-and-vegetables-based-foods       0.91      0.75      0.83      2774
                                                                                      en:desserts       0.90      0.60      0.72      1742
                                                                            en:fruits-based-foods       0.94      0.79      0.86      1590
                                                                                      en:compotes       0.94      0.76      0.84       311
                          

/home/raphael/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/raphael/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/raphael/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [7]:
cat_threshold = 0.7
precision_scores = precision_score(y_test, y_pred, average=None)
selected_class_bool = precision_scores >= cat_threshold

/home/raphael/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
no_cat_df = fr_df[pd.isnull(fr_df['categories_tags'])]

In [ ]:
no_cat_y_pred = classifier.predict_proba(no_cat_df)

In [21]:
print("Number of selected categories: {}".format(selected_class_bool.nonzero()[0].shape[0]))
print("Number of categories: {}".format(selected_class_bool.shape[0]))

Number of selected categories: 587
Number of categories: 687


In [22]:
no_cat_y_pred[:, np.logical_not(selected_class_bool)] = -1

In [23]:
category_taxonomy = categories.generate_category_taxonomy('categories.json')

In [24]:
y_pred_cat = []
y_pred_prob = []

for i in range(no_cat_y_pred.shape[0]):
    row = no_cat_y_pred[i]
    indexes = (row > 0.5).nonzero()[0]
    
    if len(indexes):
        cat = [selected_categories[i] for i in indexes]
        row_cat = categories.Category.find_deepest_item(cat, category_taxonomy)
        
#         if len(cat) > 1:
#             print("{} -> {}".format(cat, row_cat))
            
        y_pred_cat.append(row_cat)
        y_pred_prob.append(row.max())
    else:
        y_pred_cat.append(np.NaN)
        y_pred_prob.append(np.NaN)

In [25]:
no_cat_df['predicted_categories_tag'] = y_pred_cat
no_cat_df['predicted_categories_prob'] = y_pred_prob

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [26]:
no_cat_df

,code,url,last_modified_t,product_name,generic_name,brands_tags,categories_tags,countries_tags,ingredients_text,main_category_en,predicted_categories_tag,predicted_categories_prob
0,0000000000017,http://world-en.openfoodfacts.org/product/0000...,1529059204,Vitória crackers,NaN,NaN,NaN,en:france,,NaN,en:crackers,0.967427
1,0000000000031,http://world-en.openfoodfacts.org/product/0000...,1539464817,Cacao,NaN,NaN,NaN,en:france,,NaN,en:chocolates,0.854389
2,0000000000123,http://world-en.openfoodfacts.org/product/0000...,1535737986,Sauce Sweety chili 0%,NaN,NaN,NaN,en:france,,NaN,en:sauces,0.808355
3,0000000000291,http://world-en.openfoodfacts.org/product/0000...,1534239732,Mendiants,NaN,NaN,NaN,en:france,,NaN,NaN,NaN
4,0000000000949,http://world-en.openfoodfacts.org/product/0000...,1523440823,Salade de carottes râpées,NaN,NaN,NaN,en:france,,NaN,en:grated-carrots,0.992753
5,0000000000970,http://world-en.openfoodfacts.org/product/0000...,1520506371,Fromage blanc aux myrtilles,NaN,NaN,NaN,en:france,,NaN,en:cheeses,0.636843
6,0000000001001,http://world-en.openfoodfacts.org/product/0000...,1537766416,,NaN,NaN,NaN,en:france,,NaN,NaN,NaN
7,0000000001007,http://world-en.openfoodfacts.org/product/0000...,1538127565,Vainilla,NaN,NaN,NaN,en:france,,NaN,NaN,NaN
8,0000000001137,http://world-en.openfoodfacts.org/product/0000...,1539781578,Baguette parisien,NaN,NaN,NaN,en:france,,NaN,en:baguettes,0.848005
9,0000000001151,http://world-en.openfoodfacts.org/product/0000...,1537883538,&quot;Baguette Lyonnais&quot;,NaN,NaN,NaN,en:france,,NaN,NaN,NaN


In [27]:
export_df = no_cat_df.drop(['url', 'generic_name', 'brands_tags', 'categories_tags', 'countries_tags', 'product_name', 'ingredients_text', 'main_category_en'], axis=1)
export_df = export_df[pd.notnull(export_df.predicted_categories_tag)]
export_df.to_json('predicted_categories.json', orient='records', lines=True)